In [1]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
from os.path import join

### Data import

In [2]:
sample = pd.read_json(join('..', 'data', 'ner', 'sample_100_pages_names_tokens.json'))

In [3]:
sample.reset_index(drop=True, inplace=True)

In [4]:
sample.columns

Index(['categories', 'date', 'description', 'files', 'id', 'location', 'title',
       'matched_name', 'matched_category', 'matched_city', 'matched_address',
       'matched_title', 'matched_title_2', 'matched_title_str',
       'matched_title_2_str'],
      dtype='object')

### Stop words

In [5]:
stopwords_sample = pd.read_json(join('..', 'data', 'index', 'stopwords-bg.json'))[259:]

In [6]:
short_stop_word = stopwords_sample[stopwords_sample[0].apply(lambda x: len(x) <= 3)][0]

### Tokenize search columns

In [7]:
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)

In [8]:
columns_for_index = ['matched_name', 'matched_title_str', 'matched_title_2_str', 'location']



sample[columns_for_index] = sample[columns_for_index].fillna("")

for col in columns_for_index:
    sample[col + '_tokens'] = sample[col].apply(tokenizer.tokenize)

sample['tokens'] = (sample['matched_name_tokens'] +
                sample['matched_title_str_tokens'] +
                sample['matched_title_2_str_tokens'] +
                sample['location_tokens'])

### TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
sample['search_tokens_text'] = sample['tokens'].apply(lambda x: ' '.join(x))

In [11]:
tfidf_vectorizer = TfidfVectorizer(stop_words=short_stop_word.tolist())
tfidf_vectorizer.fit_transform(sample['search_tokens_text'])

<3000x3280 sparse matrix of type '<class 'numpy.float64'>'
	with 23153 stored elements in Compressed Sparse Row format>

### Cosine similarity on corpus

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
train_corpus = tfidf_vectorizer.transform(sample['search_tokens_text'])

Look for similarity betweet each document

In [14]:
duplicates = cosine_similarity(train_corpus, train_corpus)

In [15]:
duplicates

array([[1.        , 0.01394738, 0.00175835, ..., 0.00559053, 0.00738054,
        0.03803982],
       [0.01394738, 1.        , 0.00453168, ..., 0.02399727, 0.01902139,
        0.00507922],
       [0.00175835, 0.00453168, 1.        , ..., 0.00181643, 0.11174247,
        0.01313839],
       ...,
       [0.00559053, 0.02399727, 0.00181643, ..., 1.        , 0.00762435,
        0.06245791],
       [0.00738054, 0.01902139, 0.11174247, ..., 0.00762435, 1.        ,
        0.0167978 ],
       [0.03803982, 0.00507922, 0.01313839, ..., 0.06245791, 0.0167978 ,
        1.        ]])

Filter pairs with similarity >= 0.9

In [16]:
dup_pairs = []

for i in range(len(sample)):
    for j in range(len(sample)):
        if duplicates[i, j] >= 0.9 and i < j:
            dup_pairs.append((i, j))

In [17]:
dup_pairs

[(3, 2605),
 (4, 621),
 (4, 1395),
 (4, 1952),
 (4, 2490),
 (10, 1433),
 (14, 2483),
 (16, 2388),
 (17, 1366),
 (17, 2794),
 (19, 1236),
 (30, 237),
 (30, 2413),
 (32, 1902),
 (33, 2261),
 (33, 2729),
 (36, 107),
 (36, 2182),
 (36, 2936),
 (45, 133),
 (45, 259),
 (45, 1169),
 (45, 1324),
 (48, 1020),
 (48, 1318),
 (53, 582),
 (54, 168),
 (54, 279),
 (54, 586),
 (54, 769),
 (54, 1260),
 (54, 1417),
 (54, 1476),
 (54, 1721),
 (54, 2005),
 (54, 2553),
 (55, 285),
 (55, 539),
 (55, 555),
 (55, 590),
 (55, 898),
 (55, 1184),
 (55, 1287),
 (55, 1475),
 (55, 1616),
 (55, 1800),
 (55, 1894),
 (55, 1974),
 (55, 2109),
 (55, 2179),
 (55, 2192),
 (55, 2467),
 (55, 2535),
 (58, 1112),
 (59, 445),
 (59, 538),
 (59, 582),
 (59, 2193),
 (59, 2635),
 (59, 2798),
 (59, 2858),
 (63, 780),
 (65, 384),
 (65, 614),
 (65, 1506),
 (65, 1599),
 (65, 1820),
 (65, 1845),
 (65, 2024),
 (65, 2365),
 (65, 2475),
 (65, 2558),
 (67, 2926),
 (68, 153),
 (68, 225),
 (68, 1039),
 (70, 2937),
 (75, 375),
 (75, 1045),
 (

In [18]:
for doc1, doc2 in dup_pairs:
    print(sample.iloc[doc1][['title', 'description', 'location']])
    print(sample.iloc[doc2][['title', 'description', 'location']])
    print()

title                Масово пушене в заведение Маки, град Сливен
description    На 24.11.2017, 19:30 часа, от 20 маси на 6 се ...
location                  град Сливен, бул. Цар Освободител 15 а
Name: 3, dtype: object
title                       Пушене в заведение Маки, град Сливен
description    В заведението се пуши непрекъснато. * Сигналът...
location                   град Сливен, ул. Цар Освободител 17 А
Name: 2605, dtype: object

title          Нарушаване на забраната за тютюнопушене в пица...
description    В Пицария Ветрило се пуши необезпокоявано и съ...
location                         град София, ул Орехова гора 42А
Name: 4, dtype: object
title          Пушене в ресторант-пицария Ветрило, град София...
description    име обект:  Ветрило вид обект: ресторант пицар...
location       град София, жк. Стрелбище, ул. Орехова гора № 42А
Name: 621, dtype: object

title          Нарушаване на забраната за тютюнопушене в пица...
description    В Пицария Ветрило се пуши необезпокоявано

title                  Пушене в маршрутки 27, 29, 32, град София
description    Здравейте, бих искал да подам сигнал за редовн...
location                                              град София
Name: 539, dtype: object

title                       Пушене в маршрутно такси, град София
description    В 11:30 часа на 24.02.2016 г. в маршрутно такс...
location                                              град София
Name: 55, dtype: object
title          Редовно пушене в маршрутки 27, 29, 32, град София
description    Здравейте, бих искал да подам сигнал за редовн...
location                                              град София
Name: 555, dtype: object

title                       Пушене в маршрутно такси, град София
description    В 11:30 часа на 24.02.2016 г. в маршрутно такс...
location                                              град София
Name: 55, dtype: object
title              Тютюнопушене в подлези на метрото, град София
description    Здравейте, Бих искала да подам сигнал за

title          Пушене в кафе "So bar ", град София (мобилен с...
description    име обект: "So bar" вид обект: Кафе град: Софи...
location                      град София, ул. Софийски герой № 3
Name: 1205, dtype: object

title          Нарушение на забраната за тютюнопушене в Студе...
description    Здравейте, искам да съобщя, че на територията ...
location                             град София, Студентски град
Name: 99, dtype: object
title                     Пушене в градски транспорт, град София
description    Тази сутрин, в часовия диапазон 8.40-9.30, в м...
location                             град София, Студентски град
Name: 1174, dtype: object

title          Пушене в ресторант  Sweet bar and grill , град...
description    име обект: Sweet bar and grill вид обект:  Рес...
location         град София , ул. Атанас Узунов, зала Фестивална
Name: 105, dtype: object
title          пушене в ресторант "sweet bar and grill", град...
description    име обект: sweet bar and grill вид об

Name: 154, dtype: object
title                     Тютюнопуше в клуб Грамофон, град София
description    Здравейте, в момента се намирам в клуб Грамофо...
location                          град София, ул. "Будапеща" № 6
Name: 809, dtype: object

title                         Пушене в клуб Грамофон, град София
description    В клуб Грамофон в гр. София се пуши активно. А...
location                            град София, ул. Будапеща № 6
Name: 154, dtype: object
title                                       Клуб Грамофон, софия
description    Е клуб Грамофон София се пуши съвсем официално...
location                                   София, ул. Будапеща 6
Name: 1401, dtype: object

title          Пушене в бар Кабаре, град София  (мобилен сигнал)
description    име обект: Кабаре вид обект: Бар град: гр. Соф...
location                        град София, ул. Христо Белчев 12
Name: 164, dtype: object
title          Пушене в бар клуб "Кабаре", град София (мобиле...
description    име обект: К

title                 Тютюнопушене в пицария Маестро, град Варна
description    В пицария Маестро в гр. Варна, кв. Младост се ...
location       град Варна, кв. Младост, парк Младост м/у бл.1...
Name: 904, dtype: object

title                      Пуши се в пицария Маестро, град Варна
description    Днес м/у 13 и 14 ч обядвах в пицария Маестро в...
location           град Варна, парк Младост, между бл. 101 и 114
Name: 256, dtype: object
title                 Тютюнопуше в пицария "Маестро", град Варна
description    Два дни подред, обядваме в пицария Маестро, в ...
location             Варна, кв. Младост, между бл. 101 и бл. 114
Name: 1342, dtype: object

title          Пушене в ресторант "Кафана Стефанович", град С...
description    В ресторант "Кафана Стефанович" на ул. "Златов...
location                             град София, ул. Златовръх 4
Name: 259, dtype: object
title          Пушене в ресторант "Кафана Стефанович", град С...
description    име обект: Кафана Стефанович вид обек

...

Name: 2764, dtype: object

title          Сигнал за тютюнопушене в "Black label", град С...
description    Столичното заведение "Black label" в центъра, ...
location                 град София, бул. "Цар Освободител"  № 7
Name: 1772, dtype: object
title                    Пушене в обществения транспорт на София
description    Здравейте. За жалост веднага след последното м...
location                        град София, бул. Цар Освободител
Name: 2476, dtype: object

title          Неспазване на забраната за пушене в Swinging H...
description    Здравейте! Тази събота бяхме излезле с приятел...
location                     град София, ул. "Драган Цанков" № 8
Name: 1787, dtype: object
title          Нарушение на забраната за тютюнопушене на закр...
description    На 17 май 2014 г. вечерта между часовете 23:30...
location                    град София, бул. "Драган Цанков" № 8
Name: 2031, dtype: object

title          Пушене в ресторант - бирария  "City Pub ", гра...
description    име обе

title                                 Пушене в такси, град София
description    Шофьорът си позволи да изпуши няколко цигари п...
location                                              град София
Name: 1894, dtype: object
title                                Пушене в тролей, град София
description    Нарушението е в тролей 5, номер отстрани 1623....
location                                              град София
Name: 2467, dtype: object

title                                 Пушене в такси, град София
description    Шофьорът си позволи да изпуши няколко цигари п...
location                                              град София
Name: 1894, dtype: object
title                       Тютюнопушене в маршрутка, град София
description    Налага ми се всеки вторник да пътувам  до НБУ ...
location                                              град София
Name: 2535, dtype: object

title              Пушене в кафе аперитив "Йоана", град Козлодуй
description    В това заведение се пуши най-споко

title          Пушене в кафе-аперитив/магазин "Рико-33 ЕООД",...
description    име обект: Рико-33 ЕООД вид обект: Кафе-аперит...
location       село Ракита, област Плевен, ул. ”Георги Димитр...
Name: 2124, dtype: object
title          Пушене в кафе-аперитив и магазин "РИКО-33 ЕООД...
description    име обект: РИКО-33 ЕООД вид обект: Кафе-аперит...
location       село Ракита, област Плевен, ул. Георги Димитро...
Name: 2395, dtype: object

title          Пушене в Bar and dinner Sweet bar and grill , ...
description    име обект: Sweet bar and grill вид обект: Bar ...
location             град София , Кв. Гео Милев, зала Фестивална
Name: 2141, dtype: object
title            Нагло пушене в Sweet Bar and Grill , град София
description    В заведението Sweet Bar and Grill на зала Фест...
location                             град София, Зала Фестивална
Name: 2217, dtype: object

title          нарушение на забраната за тютюнопушене в Oscar...
description    На 29.12.14г. след 22.00 часа в ст

title          Пушене в пиано бар Камино, град София (мобилен...
description    име обект: Камино вид обект: Пиано бар град: С...
location                      град София, ул. „Неофит Рилски“ 70
Name: 2475, dtype: object
title                Тютюнопушене в пиано бар Камино, град София
description    Всеки 3ти човек в заведението пуши цигари и ве...
location                        град София, ул. Неофит Рилски 70
Name: 2508, dtype: object

title          Пушене в пиано бар Камино, град София (мобилен...
description    име обект: Камино вид обект: Пиано бар град: С...
location                      град София, ул. „Неофит Рилски“ 70
Name: 2475, dtype: object
title          Пушене в пиано бар Камино, град София (мобилен...
description    име обект: Камино вид обект: Пиано бар град: С...
location                        град София, ул. Неофит Рилски 70
Name: 2558, dtype: object

title          Пушене в пиано бар Камино, град София (мобилен...
description    име обект: Камино вид обект: Пиано

title          Пушене в ресторант Чорбаджии, град Пловдив (мо...
description    име обект: Чорбаджии вид обект: Ресторант град...
location                    град Пловдив, бул. Никола Вапцаров 7
Name: 2897, dtype: object

title                   Пушене в Happy Sandanski, град Сандански
description    Кога ще спре пушенето в Хепи Сандански. Бях в ...
location                     град Сандански, местност "БАЗЕНИКА"
Name: 2780, dtype: object
title                    Пушене в Хепи Сандански, град Сандански
description    В Хепи Сандански постоянно се пуши! Вече ми пи...
location                     град Сандански, местност "БАЗЕНИКА"
Name: 2964, dtype: object

title          Неспазване на забраната за тютюнопушене в рест...
description    Бях на рожден ден(пред деня) в ресторант Ниага...
location                                град София, ул. Дойран 2
Name: 2802, dtype: object
title          Неспазване на забраната за тютюнопушене в рест...
description    В ресторант Ниагара си пуши съвсем

Look for similar documnet for a given document in the corpus

In [19]:
def query_group(df_ix):
    group = cosine_similarity(train_corpus[df_ix], train_corpus)[0]
    
    dup_tuples = []
    dub_ids = []

    for j in range(3000):
        if group[j] >= 0.9:
            dup_tuples.append(j)
            
    for ix in dup_tuples:
        dub_ids.append(sample.iloc[ix]['id'])
            
    return dub_ids

In [20]:
similar_ids = query_group(1952)

In [21]:
sample[sample.id.isin(similar_ids)][['title', 'description', 'location']]

,title,description,location
4,Нарушаване на забраната за тютюнопушене в пица...,В Пицария Ветрило се пуши необезпокоявано и съ...,"град София, ул Орехова гора 42А"
621,"Пушене в ресторант-пицария Ветрило, град София...",име обект: Ветрило вид обект: ресторант пицар...,"град София, жк. Стрелбище, ул. Орехова гора № 42А"
1395,"пушене в Ресторант пицария Ветрило , София (мо...",име обект: Ветрило вид обект: Ресторант пицари...,"София, Кв . Стрелбище, Ул. Орехова гора 42а"
1952,"пушене в пицария ресторант ""Ветрило"", град Соф...",име обект: Ветрило вид обект: пицария ресторан...,"град София, Ул. Орехова гора 42А"


In [22]:
similar_ids_2 = query_group(213)

In [23]:
sample[sample.id.isin(similar_ids_2)][['title', 'description', 'location']]

,title,description,location
213,Неспазване на забраната за тютюнопушене н пица...,В ресторанта се пуши непрекъснато. * Сигналът ...,"град София, ул. Солун 43"
679,"Масово тютюнопушене в пицария ""Ветрило"", грд С...","В пицария ""Ветрило"" на ул. ""Солун"" в ж.к. ""Бор...","град София, ул. ""Солун"", срещу бл. 43"
2077,"Пушене в пицария ""Ветрило"", град София","Безкрайно съм възмутен от ситуацията, в която ...","град София, ул. Солун, с/у блок 43"
